# Семинар 5. Групповая обработка и слияние наборов данных в Pandas

На этом семинаре мы поработаем с данными о продажах глобального интернет-магазина, реализующего офисные товары и технику. Цель занятия - научиться использовать групповую обработку в Pandas и соединять таблицы данных.

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Загрузим данные:

In [3]:
orders = pd.read_csv('../datasets/global_superstore/orders.csv', encoding='utf-8')
orders.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   row_id          51290 non-null  int64  
 1   order_id        51290 non-null  object 
 2   order_date      51290 non-null  object 
 3   ship_date       51290 non-null  object 
 4   ship_mode       51290 non-null  object 
 5   customer_id     51290 non-null  object 
 6   customer_name   51290 non-null  object 
 7   segment         51290 non-null  object 
 8   postal_code     9994 non-null   float64
 9   city            51290 non-null  object 
 10  state           51290 non-null  object 
 11  country         51290 non-null  object 
 12  region          51290 non-null  object 
 13  market          51290 non-null  object 
 14  product_id      51290 non-null  object 
 15  category        51290 non-null  object 
 16  sub_category    51290 non-null  object 
 17  product_name    51290 non-null 

При загрузке со стандартными параметрами появилось несколько проблем:

- Столбец  почтового индекса (`postal_code`) распознано как числовой. Однако в некоторых странах почтовый индекс может содержать буквы и другие символы.

- Даты загрузились как текст

Добавим спецификацию форматов данных в команду для загрузки:

In [6]:
orders = pd.read_csv('../datasets/global_superstore/orders.csv', encoding='utf-8',
                    parse_dates=['order_date', 'ship_date'],
                    dtype={'postal_code' : str})
orders.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          51290 non-null  int64         
 1   order_id        51290 non-null  object        
 2   order_date      51290 non-null  datetime64[ns]
 3   ship_date       51290 non-null  datetime64[ns]
 4   ship_mode       51290 non-null  object        
 5   customer_id     51290 non-null  object        
 6   customer_name   51290 non-null  object        
 7   segment         51290 non-null  object        
 8   postal_code     9994 non-null   object        
 9   city            51290 non-null  object        
 10  state           51290 non-null  object        
 11  country         51290 non-null  object        
 12  region          51290 non-null  object        
 13  market          51290 non-null  object        
 14  product_id      51290 non-null  object        
 15  ca

Теперь данные загрузились правильно. Даты стали датами, а `postal_code` теперь - текстовое поле. Количество пропущенных значений не изменилось, в данных действительно во многих случаях индекс не указан.

При загрузке из текстовых файлов может возникать множество проблем из-за ошибок в данных или несоответствия форматов. При необходимости обратитесь к ресурсам:  

- справка Pandas https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
- Раздел 6.1 в книге Уэса Маккинни Python и анализ данных (см. ресурсы в LMS)

**Словарь данных:**

 - `row_id` - код строки заказа (уникальный)  
 - `order_id` - код заказа
 - `order_date` - дата размещения заказа
 - `ship_date` - дата отгрузки заказа (пусто, если еще не отгружен)
 - `ship_mode` - класс отправки
 - `customer_id` - код клиента
 - `customer_name` - имя клиента
 - `segment` - рыночный сегмент клиента
 - `postal_code` - почтовый индекс получателя
 - `city` - город получателя
 - `state` - штат получателя
 - `country` - страна получателя
 - `region` - регион получателя
 - `market` - рынок, к которому относится получатель
 - `product_id` - код товара
 - `category` - товарная категория
 - `sub_category` - товарная подкатегория
 - `product_name` - наименование товара
 - `sales` - выручка
 - `quantity` - количество единиц товара
 - `discount` - скидка
 - `profit` - валовая прибыль 
 - `shipping_cost` - затраты на доставку
 - `order_priority` - приоритет заказа
 

Данные представляют собой строки заказов, выполненных компанией. Каждая строка имеет уникальный номер `row_id` и содержит данные о том, кто, что и когда заказал, когда эта позиция была отправлена, адрес получателя. Также количество единиц, выручку, прибыль и затраты на доставку по строке. 

В одном заказе может быть несколько позиций. В этом случае в таблице будет несколько строк с одинаковым номером заказа:

In [6]:
orders.query('order_id == "AE-2015-GH4665138-42351"')

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
27486,43926,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,TEC-PH-5253,Technology,Phones,"Motorola Headset, VoIP",95.796,4,0.7,-156.564,6.73,Medium
29298,43924,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-ST-4078,Office Supplies,Storage,"Eldon Lockers, Blue",59.373,1,0.7,-118.767,5.83,Medium
33521,43922,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-AR-3463,Office Supplies,Art,"BIC Pencil Sharpener, Water Color",77.256,8,0.7,-82.584,4.08,Medium
40033,43925,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-ST-5693,Office Supplies,Storage,"Rogers File Cart, Single Width",42.489,1,0.7,-62.331,2.24,Medium
49469,43923,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-BI-3246,Office Supplies,Binders,"Avery Binder Covers, Clear",3.159,1,0.7,-4.971,1.25,Medium
49470,43927,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-BI-3249,Office Supplies,Binders,"Avery Binder Covers, Recycled",3.429,1,0.7,-3.891,1.25,Medium


## Задание 1

Во всех заданиях необходимо округлить результат до разумного количества десятичных знаков. Крупные суммы выводите в подходящих единицах (тысячи или млн долларов)

### 1.1. Сколько составляет оборот по каждой товарной категории? 
(выведите ответ в млн долларов с округлением до 1 десятичного знака).

In [23]:
orders_groupby = orders.groupby('category')
round(orders_groupby[['sales']].sum()/1000000, 1)

,sales
category,
Furniture,4.1
Office Supplies,3.8
Technology,4.7


### 1.2. Сколько товаров представлено в каждой товарной категории? 
(попробуйте реализовать двумя способами: только с помощью группировки/определения размера группы, и через агрегирующую функцию `nunique()` - число уникальных значений.

In [29]:
# только с помощью группировки/определения размера группы
orders.groupby('category').apply(lambda x: len(set(x['product_id'])))

category
Furniture           841
Office Supplies    2071
Technology          876
dtype: int64

In [27]:
# через агрегирующую функцию nunique()
orders_groupby_unique = orders.groupby('category')
orders_groupby_unique[['product_id']].nunique()

,product_id
category,
Furniture,841
Office Supplies,2071
Technology,876


### 1.3. Составьте рейтинг стран на основе прибыли от продаж.

In [316]:
# в тыс. рублей
round(orders.groupby('country')[['profit']].sum().sort_values(by='profit', ascending=False)/1000, 1)

,profit
country,
United States,286.4
China,150.7
India,129.1
United Kingdom,111.9
France,109.0
...,...
Pakistan,-22.4
Honduras,-29.5
Netherlands,-41.1


### 1.4. Выведите 3 самых популярных (по количеству заказов) товара на каждом рынке.

In [318]:
orders_groupby_count = orders.loc[:, ['market', 'product_id']]
orders_groupby_count ['count_id']=1 #добавлен столбец который просто считает кол-во продуктов (удобнее считать по строчкам)
orders_groupby_count

orders_groupby_count ['count_id_new'] = orders_groupby_count.groupby(['market', 'product_id'])['count_id'].transform('count')
orders_groupby_count2=orders_groupby_count.drop_duplicates()
orders_groupby_count2

orders_groupby_count_new=orders_groupby_count2.groupby('market').apply(lambda x: x.nlargest(3,'count_id_new'))
orders_groupby_count_new

market   product_id  count_id  count_id_new
market                                                               
Africa       30744        Africa  OFF-BI-4828         1            17
             482          Africa  OFF-ST-6033         1            16
             22901        Africa  OFF-AR-6120         1            16
Asia Pacific 20703  Asia Pacific  OFF-BI-3737         1            31
             21357  Asia Pacific  OFF-FA-6189         1            30
             1663   Asia Pacific  OFF-ST-6033         1            26
Europe       379          Europe  OFF-ST-4057         1            37
             11967        Europe  OFF-AR-6120         1            31
             12727        Europe  OFF-AR-5923         1            31
LATAM        30499         LATAM  OFF-BI-2917         1            22
             30805         LATAM  OFF-BI-3737         1            20
             30288         LATAM  OFF-FA-6194         1            19
USCA         28             USCA  OFF-FA-6129         1           227
             1064           USCA  OFF-BI-3302         1            20
             197            USCA  FUR-TA-4943         1            18

### 1.5. Сравните рынки по характеристикам заказов на них:
- Средний и медианный чек (сумма заказа)
- Среднее и медианное количество позиций в заказе
- Средняя и медианная прибыль по заказу
- Средняя и медианная стоимость доставки по заказу
- Среднее и медианное отношение стоимости доставки заказа к его сумме

Обратите внимание, что анализировать нужно именно характеристики заказов. В таблице содержатся _строки_ заказов, сначала на их основе нужно посчитать характеристики заказов.

In [52]:
orders_stat = orders.loc[:, ['country', 'order_id', 'sales', 'quantity', 'profit', 'shipping_cost']]
orders_stat2=orders_stat.groupby(['country', 'order_id'])[['sales', 'quantity', 'profit', 'shipping_cost']].sum()
orders_stat2['shipping_share']=orders_stat2['shipping_cost']/orders_stat2['sales']
orders_stat2
orders_stat_aggr = orders_stat2.groupby (['country']).agg(['mean', 'median'])
orders_stat_aggr

sales            quantity             profit            \
                      mean    median      mean median        mean    median   
country                                                                       
Afghanistan     833.589231  432.4200  8.769231    8.0  210.011538   71.1750   
Albania         388.812000  296.6400  4.000000    2.5   70.932000   31.0650   
Algeria         383.953085  175.6050  4.829787    3.0   96.877660   29.5650   
Angola          425.900000  197.8800  5.283333    4.0  108.249500   48.0900   
Argentina       301.108813  161.1000  7.874346    5.0  -97.873281  -35.7480   
...                    ...       ...       ...    ...         ...       ...   
Vietnam         538.926164  314.0046  8.190083    6.0  -14.027390   10.6815   
Western Sahara  666.360000  666.3600  4.000000    4.0  300.420000  300.4200   
Yemen           224.155636  157.6350  6.454545    5.0 -336.950727 -241.8420   
Zambia          565.989130  304.1400  5.456522    3.5  152.211522   58.6950   
Zimbabwe         87.560581   25.5780  4.418605    2.0 -126.250814  -47.5800   

               shipping_cost         shipping_share            
                        mean  median           mean    median  
country                                                        
Afghanistan        88.470385  45.240       0.101345  0.090030  
Albania            70.057000  33.980       0.153373  0.104618  
Algeria            39.834149  19.825       0.109778  0.080292  
Angola             40.506667  18.030       0.108626  0.105584  
Argentina          33.645874  15.691       0.120616  0.098636  
...                      ...     ...            ...       ...  
Vietnam            63.267273  27.950       0.116896  0.099234  
Western Sahara     72.520000  72.520       0.108830  0.108830  
Yemen              26.602727  24.020       0.141071  0.106509  
Zambia             66.830435  32.375       0.131857  0.114021  
Zimbabwe           11.726047   3.610       0.195400  0.169146  

[165 rows x 10 columns]

### 1.6. Рейтинг по относительной доле затрат на доставку
Рассчитайте для каждой строки (позиции) заказа долю затрат на доставку к выручке по этой строке. Для каждого рынка составьте рейтинг из 5 самых дорогих по стоимости доставке позиций. Чтобы их найти, посчитайте для каждой строки отличие доли затрат на доставку этой позиции к средней доле затрат на доставку для этого региона, товарной категории и класса доставки.

**Подсказка:** для подсчета отличий по группам используйте `transform()`.

In [8]:
orders_stat = orders.loc[:, ['market', 'region', 'category', 'ship_mode', 'shipping_cost', 'sales']]
orders_stat['shipping_share'] = orders_stat['shipping_cost']/orders_stat['sales']

# отличие доли затрат на доставку этой позиции к средней доле затрат на доставку для этого региона
orders_stat['delta_on_region'] = orders_stat.groupby('region')['shipping_share'].transform(lambda share: share - share.mean())

# отличие доли затрат на доставку этой позиции к средней доле затрат на доставку для товарной категории
orders_stat['delta_on_category'] = orders_stat.groupby('category')['shipping_share'].transform(lambda share: share - share.mean())

# отличие доли затрат на доставку этой позиции к средней доле затрат на доставку для класса доставки
orders_stat['delta_on_ship_mode'] = orders_stat.groupby('ship_mode')['shipping_share'].transform(lambda share: share - share.mean())

orders_stat



,market,region,category,ship_mode,shipping_cost,sales,shipping_share,delta_on_region,delta_on_category,delta_on_ship_mode
0,USCA,Central US,Technology,First Class,40.770,221.980,0.183665,0.030197,0.073245,-0.001928
1,Asia Pacific,Oceania,Furniture,Second Class,923.630,3709.395,0.248997,0.134282,0.138050,0.113029
2,Asia Pacific,Oceania,Technology,First Class,915.490,5175.171,0.176900,0.062185,0.066480,-0.008693
3,Europe,Western Europe,Technology,First Class,910.160,2892.510,0.314661,0.202441,0.204241,0.129067
4,Africa,Western Africa,Technology,Same Day,903.040,2832.960,0.318762,0.164290,0.208342,0.130002
...,...,...,...,...,...,...,...,...,...,...
51285,Asia Pacific,Eastern Asia,Office Supplies,Same Day,1.010,65.100,0.015515,-0.099251,-0.115327,-0.173245
51286,USCA,Southern US,Furniture,Standard Class,1.930,16.720,0.115431,-0.024993,0.004484,0.017845
51287,USCA,Western US,Office Supplies,Second Class,1.580,8.560,0.184579,0.052249,0.053737,0.048611
51288,LATAM,South America,Office Supplies,Standard Class,1.003,13.440,0.074628,-0.046535,-0.056214,-0.022958


In [12]:
# рейтинг из 5 самых дорогих по стоимости доставке позиций.
# среднее значение shipping_share у каждого рынка
orders_stat.groupby('market')[['shipping_share']].mean().sort_values(by='shipping_share', ascending=False)

,shipping_share
market,
USCA,0.140397
Africa,0.133487
Asia Pacific,0.118333
LATAM,0.117917
Europe,0.113568


### 1.7. Сравните динамику продаж по годам различных товарных подкатегорий категории `Technology`.

Попробуйте два способа: через `pivot_table()` и через `unstack()`.

**Подсказка**: компоненты даты - например, год, месяц можно получить на основе столбца с датами с помощью атрибутов `dt.year`, `dt.month`, `dt.day`:

sub_category  order_date
Accessories   2012-01-03      69.3000
              2012-01-04     260.4000
              2012-01-06      28.3200
              2012-01-07     273.7020
              2012-01-09     176.6808
                              ...    
Phones        2015-12-27     921.7680
              2015-12-28     164.3880
              2015-12-29     501.4920
              2015-12-30     730.4880
              2015-12-31    1387.8108
Name: sales, Length: 4009, dtype: float64

In [334]:
# создаем таблицу только с категорией Technology
technology = orders.query('category == "Technology"')[['sub_category', 'order_date', 'sales']]

# преобразуем дату к году
technology['order_date_year'] = technology['order_date'].apply(lambda x: x.year)

# делаем pivot table и суммируем sales у каждого года 
technology.pivot_table(['sales'], index = 'sub_category', columns = 'order_date_year', aggfunc='sum')

sales                                         
order_date_year          2012         2013          2014          2015
sub_category                                                          
Accessories      113456.00760  172397.6850  209895.16230  253488.16360
Copiers          216367.89214  327168.5988  415514.88858  550384.89376
Machines         160545.68140  159858.9511  198376.19870  260279.23590
Phones           337282.48020  364016.2744  453519.00450  552006.38010


### 1.8. Сравните динамику продаж по годам товаров брендов Apple и Samsung

In [162]:
# убираем все, кроме Apple и Samsung
samsung_and_apple = orders[orders['product_name'].str.startswith('Samsung') | orders['product_name'].str.startswith('Apple')]

# меняем имя модели на Apple или Samsung
samsung_and_apple['product_name'] = samsung_and_apple['product_name'].apply(lambda x: 'Sumsung' if x.startswith('S') else 'Apple')

# создаем поле year
samsung_and_apple['order_date_year'] = samsung_and_apple['order_date'].apply(lambda x: x.year)

# суммируем все заказы по годам
years_sales = samsung_and_apple[['product_name', 'sales', 'order_date_year']].groupby(['product_name', 'order_date_year'])[['sales']].sum()

# создаем pivot table
years_sales.pivot_table(index='product_name', columns='order_date_year')


sales                                     
order_date_year        2012        2013        2014         2015
product_name                                                    
Apple            55043.0231  55651.0190  75997.9306  100064.8754
Sumsung          64614.7703  77837.1402  75650.7963   95905.7000

### 1.9. Составьте рейтинг стран по прибыли.
В списке должны быть только страны, в которые отправлено более 500 заказов.

In [336]:
# убираем все "непопулярные" страны
orders_with_popular_countries = orders.groupby('country').filter(lambda country: len(set(country['order_id'])) > 500)

# сортируем страны по сумме profit
counties_rating = orders_with_popular_countries.groupby('country')[['profit']].sum().sort_values(by='profit', ascending=False)

# в тыс. руб.
round(counties_rating/1000, 1)

,profit
country,
United States,286.4
China,150.7
India,129.1
United Kingdom,111.9
France,109.0
Germany,107.2
Australia,103.9
Mexico,102.5
Brazil,29.7


## Задание 2

Дополним данные сведениями о том, какие заказы были возвращены.

In [165]:
returns = pd.read_csv('../datasets/global_superstore/returns.csv')
returns.head()

,returned,order_id,region
0,True,CA-2012-SA20830140-41210,Central US
1,True,IN-2012-PB19210127-41259,Eastern Asia
2,True,CA-2012-SC20095140-41174,Central US
3,True,IN-2015-JH158207-42140,Oceania
4,True,IN-2014-LC168857-41747,Oceania


In [166]:
returns.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 3 columns):
returned    1079 non-null bool
order_id    1079 non-null object
region      1079 non-null object
dtypes: bool(1), object(2)
memory usage: 18.0+ KB


В таблице `returns` содержатся записи о тех заказах, которые были возвращены - номер заказа, регион и статус возврата.

Довольно необычной особенностью этого набора данных является то, что номера заказов в нем не уникальны. Заказы с одинаковым номером могли быть зарегистрированы в разных регионах:

In [13]:
(orders[['order_id', 'region']]
 .groupby('order_id',)
 .filter(lambda grp: grp['region'].nunique() >1)
 .reset_index(drop=True))

,order_id,region
0,CA-2015-AG10330140-42361,Western US
1,CA-2015-AG10330140-42361,Central US
2,CA-2015-AG10330140-42361,Central US
3,CA-2015-AG10330140-42361,Central US
4,CA-2014-BW11200140-41940,Eastern US
5,CA-2014-BW11200140-41940,Southern US
6,CA-2014-BW11200140-41940,Eastern US
7,CA-2014-BW11200140-41940,Eastern US
8,CA-2014-BW11200140-41940,Eastern US
9,CA-2012-CS12355140-41135,Southern US


По этой причине соединять таблицы необходимо по двум ключам - `order_id` и `region`.

Попробуем соединить таблицы: 

In [187]:
orders_returns = orders.merge(returns, on=['order_id', 'region']) # лучше явно указать ключи
orders_returns.shape

(2202, 25)

Строк получилось очень мало. В исходном наборе было более 50 тысяч строк:

### 2.1. Почему пропали строки? 
Исправьте аргументы `merge()`, чтобы задать правильный тип соединения.

In [313]:
# нужно указать другой вид соединения
orders_returns = orders.merge(returns, on=['order_id', 'region'], how='left')
orders_returns.shape

(51290, 25)

### 2.2. Составьте рейтинг регионов по убыванию доли возвращенных заказов.

In [314]:
# так как True это 1 а False это 0, то количество возвратов это просто сумма ряда returned
amount_of_returned_orders = orders_returns.groupby('region')['returned'].sum()
amount_of_orders = orders_returns.groupby('region').size()

returned_share = amount_of_returned_orders / amount_of_orders
returned_share.name = 'Доля возвращенных заказов'
returned_share.sort_values(ascending=False).to_frame()


,Доля возвращенных заказов
region,
Western US,0.055261
Eastern Asia,0.055181
Southern Europe,0.053005
Southern Africa,0.052301
Southern US,0.051235
Eastern US,0.047051
Southeastern Asia,0.044743
South America,0.044511
Western Asia,0.044262


### 2.3. Рейтинг менеджеров

В еще одной таблице - `people` - содержатся данные о менеджерах, которые руководят продажами в различных регионах:

In [237]:
people = pd.read_csv('../datasets/global_superstore/people.csv')
people.head()

,person,region
0,Marilène Rousseau,Caribbean
1,Andile Ihejirika,Central Africa
2,Nicodemo Bautista,Central America
3,Cansu Peynirci,Central Asia
4,Lon Bonher,Central US


In [307]:
# в таблице все регионы уникальны
people.region.nunique() == len(people.region)

True

Соедините таблицу `orders` с этими данными и выведите рейтинг менеджеров по прибыли.

In [306]:
orders_with_persons = orders.merge(people, on=['region'], how='left')

# на всякий случай посмотрим у каких регионов нет менеджера
orders_with_persons[orders_with_persons['person'].isna()][['region']].groupby('region').groups.keys() # == ['Canada'] значит только у Канады, не страшно

# в тыч. руб.
round(orders_with_persons.groupby('person')[['profit']].sum().sort_values(by='profit',ascending=False)/1000, 1)

,profit
person,
Gilbert Wolff,218.4
Hadia Bousaid,167.1
Chandrakant Chaudhri,159.3
Nicodemo Bautista,159.0
Kauri Anaru,120.1
Derrick Snyders,108.4
Dolores Davis,91.5
Miina Nylund,83.9
Oxana Lagunov,77.1


### 2.4. Объединение таблиц
На основе таблицы `orders` определите три самые прибыльные товарные подкатегории на рынке США и сохраните их в таблице `best_usa`. Затем на основе этой же таблицы определите три самые прибыльные подкатегории на рынке Европы и сохраните их в таблице `best_europe`. Затем объедините данные из таблиц `best_usa` и `best_europe` в одной таблице.

In [300]:
# уберем из таблицы все кроме US
usa_only = orders[orders['region'].str.contains('US')]
# первые 3 по profit
best_usa = usa_only.groupby('sub_category')[['profit']].sum().nlargest(3, 'profit')

# уберем из таблицы все кроме Europe
europe_only = orders[orders['region'].str.contains('Europe')]
# первые 3 по profit
best_europe = europe_only.groupby('sub_category')[['profit']].sum().nlargest(3, 'profit') 

# изменим имя для удобства
best_usa = best_usa.rename({'profit':'USA profit'}, axis='columns')
best_europe = best_europe.rename({'profit': 'Europe profit'}, axis='columns')



In [303]:
# разные виды объединения
pd.concat([best_usa, best_europe], axis='columns')

,Europe profit,USA profit
sub_category,,
Copiers,NaN,55617.8249
Phones,NaN,44515.7306
Accessories,NaN,41936.6357
Bookcases,68138.595,NaN
Copiers,62994.864,NaN
Appliances,52963.110,NaN


In [302]:
pd.concat([best_usa.reset_index(), best_europe.reset_index()], axis='columns')

,sub_category,USA profit,sub_category,Europe profit
0,Copiers,55617.8249,Bookcases,68138.595
1,Phones,44515.7306,Copiers,62994.864
2,Accessories,41936.6357,Appliances,52963.110


In [304]:
pd.concat([best_usa, best_europe], axis='rows')

,Europe profit,USA profit
sub_category,,
Copiers,NaN,55617.8249
Phones,NaN,44515.7306
Accessories,NaN,41936.6357
Bookcases,68138.595,NaN
Copiers,62994.864,NaN
Appliances,52963.110,NaN
